# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load exported csv from WeatherPy
weather_data_file = pd.read_csv("../WeatherPy/weatherdf.csv")
weather_data_file

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Thinadhoo,0.53,72.93,84.97,68,4,6.15,MV,1594695892
1,Xam Nua,20.42,104.04,86.86,45,1,1.03,LA,1594695918
2,Qaanaaq,77.48,-69.36,42.33,70,1,3.60,GL,1594695918
3,Spring Hill,28.48,-82.53,82.00,88,1,7.61,US,1594695919
4,Puqi,29.72,113.88,85.26,73,100,4.27,CN,1594695919
...,...,...,...,...,...,...,...,...,...
552,Shingū,33.73,135.98,77.00,100,75,10.29,JP,1594696057
553,Honningsvåg,70.98,25.97,46.40,75,75,8.05,NO,1594695763
554,George Town,5.41,100.34,84.20,83,40,4.70,MY,1594695805
555,Beloha,-25.17,45.05,57.47,93,20,2.64,MG,1594695761


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [19]:
locations = weather_data_file[["Latitude", "Longitude"]]
humidity = weather_data_file["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity=300)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
ideal_city = weather_data_file.loc[(weather_data_file["Max Temp"] > 75) & 
                                   (weather_data_file["Wind Speed"] < 5) & (weather_data_file["Cloudiness"] == 0)].dropna()
ideal_city

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,Olhão,37.03,-7.84,77.00,78,0,3.36,PT,1594695996
158,Mamlyutka,54.94,68.54,77.00,69,0,2.24,KZ,1594696005
232,Ürümqi,43.80,87.60,80.60,28,0,4.47,CN,1594695788
321,Suntar,62.14,117.63,80.73,42,0,3.60,RU,1594695879
323,Portland,45.52,-122.68,77.00,45,0,1.01,US,1594695433
350,Kachug,53.96,105.88,81.09,48,0,4.38,RU,1594696032
449,Jiuquan,39.74,98.52,83.28,28,0,4.18,CN,1594696044
534,Kumluca,36.37,30.29,83.03,52,0,2.48,TR,1594696055


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = ideal_city[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
94,Olhão,PT,37.03,-7.84,
158,Mamlyutka,KZ,54.94,68.54,
232,Ürümqi,CN,43.80,87.60,
321,Suntar,RU,62.14,117.63,
323,Portland,US,45.52,-122.68,
350,Kachug,RU,53.96,105.88,
449,Jiuquan,CN,39.74,98.52,
534,Kumluca,TR,36.37,30.29,


In [15]:
# geocoordinates
#target_coordinates = ideal_city["City"]
#target_search = "Chinese"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in ideal_city.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    params["location"] = f"{lat},{long}"

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    hotel_info = response.json()
    try:
        ideal_city.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
    except (KeyError, IndexError):
        print("Next Hotel")

#print(hotel_info)
    

Next Hotel


In [16]:
ideal_city

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
94,Olhão,37.03,-7.84,77.00,78,0,3.36,PT,1594695996,Real Marina Hotel & Spa
158,Mamlyutka,54.94,68.54,77.00,69,0,2.24,KZ,1594696005,NaN
232,Ürümqi,43.80,87.60,80.60,28,0,4.47,CN,1594695788,Hua Ling Grand Hotel Urumqi
321,Suntar,62.14,117.63,80.73,42,0,3.60,RU,1594695879,Gostevoy Dom
323,Portland,45.52,-122.68,77.00,45,0,1.01,US,1594695433,Portland Marriott City Center
350,Kachug,53.96,105.88,81.09,48,0,4.38,RU,1594696032,"The hotel ""Empire"""
449,Jiuquan,39.74,98.52,83.28,28,0,4.18,CN,1594696044,Jiuquan Hotel
534,Kumluca,36.37,30.29,83.03,52,0,2.48,TR,1594696055,TurkuazKöy


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
# fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))